In [2]:
# Librerías necesarias
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
import pandas as pd
from conexion import get_engine
from sqlalchemy import text


In [3]:
# Conectar a una base de datos
engine = get_engine("basedeprueba")
# Mostrar el nombre de la base de datos conectada
print(engine.url.database)

basedeprueba


In [ ]:
# Crear una base de datos nueva
instruccion = text("CREATE DATABASE nombre_de_la_nueva_base")

with engine.connect().execution_options(isolation_level="AUTOCOMMIT") as conn:
        conn.execute(instruccion)    
    

In [23]:
# Mostrar las bases de datos existentes (en PostgreSQL, excluyendo las plantillas)
query = "SELECT datname FROM pg_database WHERE datistemplate = false"

df = pd.read_sql(query, engine)
display(df.head())

,datname
0,postgres
1,retaildw
2,sqlda
3,basedeprueba


In [ ]:
# Eliminar una base de datos existente
instruccion = text("DROP DATABASE IF EXISTS nombre_de_la_base_a_eliminar")

with engine.connect().execution_options(isolation_level="AUTOCOMMIT") as conn:
        conn.execute(instruccion)       

In [4]:
# Crear una tabla nueva
instruccion = text("""
    CREATE TABLE IF NOT EXISTS empleados (
        id SERIAL PRIMARY KEY,
        nombre VARCHAR(100) NOT NULL,
        id_jefe INT,
        FOREIGN KEY (id_jefe) REFERENCES empleados(id)        
    )
""")

with engine.connect() as conn:
        conn.execute(instruccion)
        conn.commit()

In [5]:
# Listar todas las tablas de la base de datos conectada
query = """
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'public'
"""
df = pd.read_sql(query, engine)
display(df)

,table_name
0,alumnos
1,empleados


In [6]:
# Insertar datos en la tabla empleados
instruccion = text("""
    INSERT INTO empleados (id, nombre, id_jefe) VALUES
    (1, 'Ana', NULL),
    (2, 'Luis', 1),
    (3, 'Marta', 1),
    (4, 'Jorge', 2),
    (5, 'Diana', 2),
    (6, 'Pedro', 4);
""")

with engine.connect() as conn:
        conn.execute(instruccion)
        conn.commit()

In [4]:
query = "SELECT * FROM empleados"
df = pd.read_sql(query, engine)
# display(df.style.hide())
print(df.to_string(index=False))

 id nombre  id_jefe
  1    Ana      NaN
  2   Luis      1.0
  3  Marta      1.0
  4  Jorge      2.0
  5  Diana      2.0
  6  Pedro      4.0
